In [2]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context
from flickrapi import FlickrAPI
from urllib.request import urlretrieve
from pprint import pprint
import os, time, sys

key = "6374acf41040d1dbc864d9dd5f8718e2"
secret = "9770f7c6e822dfd6"
wait_time = 1

animal_list = ['capybara', 'cat', 'otter']
for animalname in animal_list:
    savedir = './' + animalname

flickr = FlickrAPI(key, secret, format='parsed-json')
result = flickr.photos.search(
    text = animalname,
    per_page = 400,
    media = 'photos',
    sort = 'relevance',
    safe_search = 1,
    extras = 'url_q, lisence'
)

photos = result['photos']
# pprint(photos)

for i, photo in enumerate(photos['photo']):
    url_q = photo['url_q']
    filepath = savedir + '/' + photo['id'] + '.jpg'
    if os.path.exists(filepath): continue
    urlretrieve(url_q, filepath)
    time.sleep(wait_time)


In [3]:
from PIL import Image
import os, glob
import numpy as np
from sklearn import model_selection

classes = ['capybara', 'cat', 'otter']
num_classes = len(classes)
image_size = 50

X = []
Y = []
for index, classlabel in enumerate(classes):
    photos_dir = './' + classlabel
    files = glob.glob(photos_dir + '/*.jpg')
    for i, file in enumerate(files):
        if i >= 200: break
        image = Image.open(file)
        image = image.convert('RGB')
        image = image.resize((image_size, image_size))
        data = np.asarray(image)
        X.append(data)
        Y.append(index)

X = np.array(X)
Y = np.array(Y)

X_train, X_test, Y_train, Y_test = model_selection.train_test_split(X, Y)
xy = (X_train, X_test, Y_train, Y_test)
np.save('./animal.npy', xy)


In [5]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.utils import np_utils
import keras
import numpy as np

classes = ['capybara', 'cat', 'otter']
num_classes = len(classes)
image_size = 50

#メイン関数の定義
def main():
    X_train, X_test, Y_train, Y_test = np.load('./animal.npy')
    X_train = X_train.astype('float') / 256
    X_test = X_test.astype('float') / 256
    Y_train = np_utils.to_categorical(Y_train, num_classes)
    Y_test = np_utils.to_categorical(Y_test, num_classes)

    model = model_train(X_train, Y_train)
    model_eval(model, X_test, Y_test)

def model_train(X,Y):
    model = Sequential()
    model.add(Conv2D(32,(3,3), padding='same', input_shape=X.shape[1:]))
    model.add(Activation('relu'))
    model.add(Conv2D(32,(3,3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(64, (3,3), padding='same'))
    model.add(Activation('relu'))
    model.add(Conv2D(64, (3,3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(512))
    model.add(Dropout(0.5))
    model.add(Dense(3))
    model.add(Activation('softmax'))

    opt = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)

    model.compile(loss='categorical_crossentropy',
                  optimizer=opt, metrics=['accuracy'])

    model.fit(X,Y,batch_size=32, nb_epoch=100)

    model.save('./animal_cnn.h5')

    return model

def model_eval(model, X, Y):
    scores = model.evaluate(X,Y,verbose=1)
    print('Test Loss:',scores[0])
    print('Test Accuracy:', scores[1])

if __name__ == '__main__':
    main()


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:50: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


Epoch 1/100


 32/450 [=>............................] - ETA: 9s - loss: 1.1352 - acc: 0.3125

 64/450 [===>..........................] - ETA: 6s - loss: 1.2903 - acc: 0.2812

 96/450 [=====>........................] - ETA: 4s - loss: 1.2422 - acc: 0.3229

...

 64/150 [===========>..................] - ETA: 0s

 96/150 [==================>...........] - ETA: 0s

128/150 [========================>.....] - ETA: 0s

150/150 [==============================] - 0s 3ms/step


Test Loss: 1.477488423983256
Test Accuracy: 0.5866666650772094


In [7]:

from keras.models import Sequential, load_model
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
# from keras.utils import np_utils
import keras,sys
import numpy as np
from PIL import Image

classes = ["capybara","cat","otter"]
num_classes = len(classes)
image_size = 50

def build_model():
    model = Sequential()
    model.add(Conv2D(32,(3,3), padding='same',input_shape=(50,50,3)))
    model.add(Activation('relu'))
    model.add(Conv2D(32,(3,3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(64,(3,3), padding='same'))
    model.add(Activation('relu'))
    model.add(Conv2D(64,(3,3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(512))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(3))
    model.add(Activation('softmax'))

    opt = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)

    model.compile(loss='categorical_crossentropy',optimizer=opt,metrics=['accuracy'])

    # モデルのロード
    model = load_model('./animal_cnn.h5')

    return model

def main():
    image = Image.open('./capybara/11517231.jpg')
    image = image.convert('RGB')
    image = image.resize((image_size, image_size))
    data = np.asarray(image)/255
    X = []
    X.append(data)
    X = np.array(X)
    model = build_model()

    result = model.predict([X])[0]
    predicted = result.argmax()
    percentage = int(result[predicted] * 100)
    print("{0} ({1} %)".format(classes[predicted], percentage))

if __name__ == "__main__":
    main()


capybara (99 %)
